# Konwolucja binarna

##### Konfiguracja pozwalająca na manualne sprawdzenie poprawności inferencji oraz gradientów oraz porównanie ze zwykłą konwolucją

In [63]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys
import time

from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.python.framework import function

import numpy as np
import tensorflow as tf
from tensorflow.python.framework import ops
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import sparse_ops

FLAGS = None
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='VALID')
      
def conv_bin_func(func, inp, Tout, stateful=True, name=None, grad=None):
    g = tf.get_default_graph()
    with g.gradient_override_map({"PyFunc": "Conv2D"}):
        return tf.py_func(func, inp, Tout, stateful=stateful, name=name)

def mysquare(x, name=None):
    
    with ops.op_scope([x], name, "Mysquare") as name:
        sqr_x = py_func(np.square,
                        [x],
                        [tf.float32],
                        name=name,
                        grad=_MySquareGrad)  # <-- here's the call to the gradient
        return sqr_x[0]

x = tf.placeholder(tf.float32, [None, 9])
x_reshaped = tf.reshape(x, [-1,3,3,1])
w = np.zeros([2,2,1,1])
w[0:2,0:2,0,0] = np.vstack(([0.1,1],[1,1]))
W_conv1 = tf.Variable(tf.cast(w,dtype=tf.float32))
G = tf.get_default_graph()
with G.gradient_override_map({"Sign": "Identity"}):
            w_shape = tf.shape(W_conv1)
            n = tf.cast(tf.reduce_prod(w_shape[0:-1]),tf.float32) 
            abs = tf.abs(W_conv1)
            a = tf.stop_gradient(tf.reduce_sum(abs, [0,1,2])/n)  
            absX = tf.abs(x_reshaped)
            k = tf.ones(w_shape, dtype=tf.float32) / n
            K = conv2d(absX, k)
            x_sign = tf.sign(x_reshaped)
            w_sign = tf.sign(W_conv1/a)
            h_conv_bin = conv2d(x_sign, w_sign)*a*K

h_conv = conv2d(x_reshaped, W_conv1)

sess = tf.Session()
sess.as_default()
sess.run(tf.global_variables_initializer())

numbers = np.reshape(np.array([1,2,3,4,5,6,7,8,9], dtype=np.float32),(1,9))
result_bin = h_conv_bin.eval(session=sess, feed_dict={x: numbers})
result = h_conv.eval(session=sess, feed_dict={x: numbers})
print(result_bin[0,0:2,0:2,0])
print(result[0,0:2,0:2,0])

[[  9.29999924  12.39999962]
 [ 18.59999847  21.69999886]]
[[ 11.10000038  14.19999981]
 [ 20.39999962  23.5       ]]


In [64]:
var_grad_bin = tf.gradients(h_conv_bin, W_conv1)[0]
gradients_bin = var_grad_bin.eval(session=sess, feed_dict={x: np.reshape(numbers,(1,9))})
var_grad = tf.gradients(h_conv, W_conv1)[0]
gradients = var_grad.eval(session=sess, feed_dict={x: np.reshape(numbers,(1,9))})
print(gradients_bin)
print(gradients)

[[[[ 19.99999809]]

  [[ 19.99999809]]]


 [[[ 19.99999809]]

  [[ 19.99999809]]]]
[[[[ 12.]]

  [[ 16.]]]


 [[[ 24.]]

  [[ 28.]]]]
